In [17]:
import pydicom
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from sklearn.model_selection import KFold
from tqdm import tqdm

In [18]:

# Basisdirectory waar alle DICOM-afbeeldingen zijn opgeslagen
root_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'

# Lijst om alle DICOM-bestandspaden op te slaan
dicom_paths = []

# Loop door alle subdirectories (study_id en series_id)
for study_dir in os.listdir(root_dir):
    study_path = os.path.join(root_dir, study_dir)
    
    if os.path.isdir(study_path):
        for series_dir in os.listdir(study_path):
            series_path = os.path.join(study_path, series_dir)
            
            if os.path.isdir(series_path):
                # Zoek naar alle .dcm-bestanden in deze serie-directory
                dicom_files = glob.glob(os.path.join(series_path, '*.dcm'))
                
                # Voeg elk DICOM-bestandspad toe aan de dicom_paths-lijst
                dicom_paths.extend(dicom_files)

# Controleer het aantal gevonden DICOM-bestanden
print(f'Aantal gevonden DICOM-bestanden: {len(dicom_paths)}')


Aantal gevonden DICOM-bestanden: 147218


In [19]:
class SegmentationDataset(Dataset):
    def __init__(self, dicom_paths, coords_dict, target_size=(512, 512), transform=None):
        self.dicom_paths = dicom_paths
        self.coords_dict = coords_dict
        self.target_size = target_size
        self.transform = transform

    def __getitem__(self, idx):
        dicom_path = self.dicom_paths[idx]
        
        # Lees de DICOM-afbeelding
        dicom_data = pydicom.dcmread(dicom_path)
        image = dicom_data.pixel_array
        
        # Schalen naar [0, 255]
        image = (image - image.min()) / (image.max() - image.min() + 1e-6) * 255

        # Gebruik cv2.resize om de afbeelding naar de gewenste grootte te schalen
        image = cv2.resize(image, self.target_size, interpolation=cv2.INTER_CUBIC)

        # Controleer of er coördinaten zijn voor dit pad
        if dicom_path in self.coords_dict:
            coords = self.coords_dict[dicom_path]
            mask = self.generate_mask(image.shape, coords)
        else:
            return None  # Sla deze afbeelding over als er geen coördinaten beschikbaar zijn

        # Converteer de afbeelding en het masker naar tensoren
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Voeg een kanaal toe (1-kanaals)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)  # Voeg een kanaal toe (1-kanaals)

        return image, mask

    def generate_mask(self, image_shape, coords):
        # Genereer een leeg masker
        mask = np.zeros(image_shape, dtype=np.uint8)

        # Voeg cirkels toe op de coördinaten
        for (x, y) in coords:
            cv2.circle(mask, (int(x), int(y)), radius=10, color=255, thickness=-1)

        # Converteer het masker naar binaire waarden (0 of 1)
        mask = mask / 255.0  # Verdeel door 255 om de waarden te normaliseren naar 0 en 1

        return mask

    def __len__(self):
        return len(self.dicom_paths)


In [20]:
def collate_fn(batch):
    # Verwijder None-items uit de batch
    batch = [item for item in batch if item is not None]
    
    if len(batch) == 0:
        return None  # Als de batch leeg is, retourneer None
    
    return torch.utils.data.default_collate(batch)


In [21]:
rd = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
df = pd.read_csv(f'{rd}/train_series_descriptions.csv')
st_ids = df['study_id'].unique()
dfc = pd.read_csv(f'{rd}/train_label_coordinates.csv')
desc = list(df['series_description'].unique())

In [22]:
import os
import glob
from tqdm import tqdm

# Maak een dictionary aan om de coördinaten op te slaan
coords_dict = {}

# Itereer door de rijen in dfc en koppel de coördinaten aan de juiste DICOM afbeelding
for idx, row in dfc.iterrows():
    study_id = row['study_id']
    series_id = row['series_id']
    instance_number = row['instance_number']
    
    # Zoek de serie beschrijving (series_description) op
    df_series = df[(df['study_id'] == study_id) & (df['series_id'] == series_id)]
    
    if len(df_series) == 0:
#         print(f"Geen series_description gevonden voor study_id: {study_id} en series_id: {series_id}")
        continue
    
    series_desc = df_series['series_description'].values[0]
    ds_ = series_desc.replace('/', '_')  # Vervang slash om te zorgen dat pad geldig is
    
    # Genereer het pad naar de DICOM afbeelding
    dicom_filename = f'{int(instance_number):01d}.dcm'
    dicom_path = f'{rd}/train_images/{study_id}/{series_id}/{dicom_filename}'
    
    # Controleer of het bestand daadwerkelijk bestaat
    if os.path.exists(dicom_path):
        # Coördinaten (x, y)
        x, y = row['x'], row['y']
        
        # Voeg de coördinaat toe aan het corresponderende pad in de coords_dict
        if dicom_path not in coords_dict:
            coords_dict[dicom_path] = []
        
        coords_dict[dicom_path].append((x, y))
    else:
        print(f"Bestand bestaat niet: {dicom_path}, coördinaten worden overgeslagen.")
        continue

print(f"Aantal paden in coords_dict: {len(coords_dict)}")


Aantal paden in coords_dict: 24546


In [9]:
from torch import nn

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        f"{name}_conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (f"{name}_norm1", nn.BatchNorm2d(num_features=features)),
                    (f"{name}_relu1", nn.ReLU(inplace=True)),
                    (
                        f"{name}_conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (f"{name}_norm2", nn.BatchNorm2d(num_features=features)),
                    (f"{name}_relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import os
import torch

import os

def save_checkpoint(model, optimizer, epoch, fold, checkpoint_dir='checkpoints'):
    """Functie om een checkpoint op te slaan."""
    # Controleer of de directory bestaat, en maak deze indien nodig aan
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Maak het pad voor de checkpoint met de fold-naam
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_fold_{fold}.pth')  # Correct gebruik van fold in filename

    # Sla het model, optimizer, en epoch op in de checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'fold': fold
    }, checkpoint_path)
    
    print(f'Checkpoint saved: {checkpoint_path}')

    
def load_checkpoint(model, optimizer, fold, checkpoint_dir='checkpoints'):
    """Functie om een checkpoint te laden."""
    # Gebruik de fold-waarde in de bestandsnaam en converteer naar string
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_fold_{fold}.pth')
    
    if os.path.exists(checkpoint_path):
        print(f"Checkpoint gevonden: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        
        # Laad de status van het model en de optimizer
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        # Laad het epoch nummer
        start_epoch = checkpoint['epoch'] + 1
        print(f"Hervatten vanaf epoch {start_epoch}")
        
        return start_epoch
    else:
        print(f"Geen checkpoint gevonden in {checkpoint_path}, beginnen vanaf epoch 1.")
        return 1  # Start vanaf epoch 1 als er geen checkpoint is

In [ ]:
# KFold instellen (bijv. 5 folds)
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# dicom_paths en coords_dict bevatten respectievelijk de DICOM-paden en coördinaten
# dicom_paths: Lijst van paden naar DICOM-bestanden
# coords_dict: Dictionary met coördinaten voor elke DICOM-bestandspad

# Creëer de folds
dataset_size = len(dicom_paths)
indices = list(range(dataset_size))

# Zet de resultaten van alle folds op nul
results = {}

# Hyperparameters voor early stopping
early_stopping_limit = 3  # Maximaal aantal epochs zonder verbetering
early_stopping_patience = early_stopping_limit

# KFold loop
for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
    print(f'Fold {fold+1}/{k_folds}')
    
    # Verdeel dicom_paths in train en validatie sets
    train_dicom_paths = [dicom_paths[i] for i in train_idx]
    val_dicom_paths = [dicom_paths[i] for i in val_idx]
    
    # Creëer de train en validatie datasets met coords_dict voor het genereren van maskers
    train_dataset = SegmentationDataset(dicom_paths=train_dicom_paths, coords_dict=coords_dict)
    val_dataset = SegmentationDataset(dicom_paths=val_dicom_paths, coords_dict=coords_dict)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, drop_last=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, collate_fn=collate_fn)

    # Instantieer het model
    model = UNet(in_channels=1, out_channels=1).to(device)  # in_channels aangepast voor grijswaarden DICOM-afbeeldingen

    # Definieer de loss-functie en optimizer
    criterion = nn.BCELoss()  # Binary Cross-Entropy voor pixel-wise classificatie
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Controleer of er een checkpoint is en hervat de training vanaf dat punt
    start_epoch = load_checkpoint(model, optimizer, fold)
    
    # Training loop
    num_epochs = 20
    best_val_loss = float('inf')  # Initieer met een hoge waarde om het beste model op te slaan
    early_stopping_counter = 0  # Teller voor early stopping

    for epoch in range(start_epoch, num_epochs + 1):
        model.train()
        running_loss = 0.0
    
        # Trainen
        for batch in tqdm(train_loader):
            if batch is None:  # Sla lege batches over
                continue
            images, masks = batch
            images = images.to(device)
            masks = masks.to(device)
            # Voorwaartse pass
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Achterwaartse pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Gemiddelde loss van de epoch
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch}/{num_epochs}], Train Loss: {epoch_loss:.4f}')
        
        # Validatie
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                if batch is None:
                    continue
                images, masks = batch
                images = images.to(device)
                masks = masks.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        print(f'Fold {fold+1}, Epoch {epoch}, Validation Loss: {val_loss:.4f}')
        
        # Controleer op verbetering van validatie loss
        if val_loss < best_val_loss:
            print(f'Saving best model for fold {fold+1}, epoch {epoch}')
            torch.save(model.state_dict(), f'unet_best_fold_{fold+1}.pth')
            best_val_loss = val_loss
            early_stopping_counter = 0  # Reset de early stopping teller
        else:
            early_stopping_counter += 1
            print(f'No improvement for {early_stopping_counter} epochs.')
        
        # Opslaan van een checkpoint na elke epoch
        save_checkpoint(model, optimizer, epoch, fold, checkpoint_dir='checkpoints')


        # Controleer of de early stopping limiet is bereikt
        if early_stopping_counter >= early_stopping_patience:
            print(f'Early stopping in fold {fold+1} at epoch {epoch}')
            break  # Verlaat de training loop voor deze fold

    # Sla de resultaten van deze fold op
    results[fold] = best_val_loss

# Print de resultaten van elke fold
for fold in range(k_folds):
    print(f'Fold {fold+1}: Best Validation Loss: {results[fold]:.4f}')

# Gemiddelde validatie loss over alle folds
avg_val_loss = np.mean([results[fold] for fold in range(k_folds)])
print(f'Gemiddelde validatie loss over alle {k_folds} folds: {avg_val_loss:.4f}')

In [15]:
# Instantieer het model met 1 input channel, wat overeenkomt met het opgeslagen model
model_fold_1 = UNet(in_channels=1, out_channels=1).to(device)
model_fold_2 = UNet(in_channels=1, out_channels=1).to(device)

# Laad de opgeslagen checkpoints van beide folds, map naar CPU
model_fold_1.load_state_dict(torch.load('/kaggle/input/masks_models/pytorch/default/1/unet_best_fold_1 (3).pth', map_location=torch.device('cpu')))
model_fold_2.load_state_dict(torch.load('/kaggle/input/masks_models/pytorch/default/1/unet_best_fold_2.pth', map_location=torch.device('cpu')))

# Zorg ervoor dat het model op het juiste device wordt gezet
model_fold_1.to(device)
model_fold_2.to(device)

model_fold_1.eval()
model_fold_2.eval()


/tmp/ipykernel_30/3936207421.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_fold_1.load_state_dict(torch.load('/kaggle/input/masks_models/pytorch/default/1/unet_b

UNet(
  (encoder1): Sequential(
    (enc1_conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1_relu1): ReLU(inplace=True)
    (enc1_conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1_norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1_relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2_conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2_relu1): ReLU(inplace=True)
    (enc2_conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [ ]:
# Stel een lijst op van maskers en bijbehorende afbeeldingen
predicted_masks = []
train_dataset = SegmentationDataset(dicom_paths=dicom_paths, coords_dict=coords_dict)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, drop_last=True, collate_fn=collate_fn)

with torch.no_grad():
    for batch in tqdm(train_loader):  # Neem de images uit de train_loader of een specifieke loader
        if batch is None:
            continue
        images, masks = batch
        images = images.to(device)        
        # Voorspellingen van beide modellen
        outputs_fold_1 = model_fold_1(images)
        outputs_fold_2 = model_fold_2(images)
        
        # Combineer de voorspellingen (gemiddeld bijvoorbeeld)
        combined_mask = (outputs_fold_1 + outputs_fold_2) / 2
        
        predicted_masks.append(combined_mask.cpu())  # Voeg het gecombineerde masker toe aan de lijst



  1%|          | 81/9201 [09:00<21:40:43,  8.56s/it]